In [1]:
import pandas as pd
import time

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
test2=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])

In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
all_data=pd.concat([train[['qid','uid','time']],test,test2])

In [5]:
all_data['day']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))

100%|██████████| 11772563/11772563 [00:24<00:00, 490187.17it/s]


In [7]:
all_data['hour']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[1]))

100%|██████████| 11772563/11772563 [00:23<00:00, 510072.95it/s]


In [8]:
def time_(x):
    temp=x[1:].split('-H')
    return int(temp[0])*24+int(temp[1])

In [9]:
all_data['time']=all_data['time'].progress_apply(lambda x:time_(x))

100%|██████████| 11772563/11772563 [00:29<00:00, 403208.49it/s]


In [10]:
all_data['day']=all_data['day'].astype(int)
all_data['hour']=all_data['hour'].astype(int)

In [11]:
all_data['uday_invite_rank']=all_data.groupby(['uid','day'])['hour'].rank(ascending=1,method='min')
feature=all_data.groupby(['uid','day','hour'])['qid'].agg('count').reset_index()
feature.columns=['uid', 'day', 'hour', 'uday_sametime_icount']
all_data=all_data.merge(feature,on=['uid', 'day', 'hour'],how='left')

In [12]:
all_data['qday_invite_rank']=all_data.groupby(['qid','day'])['hour'].rank(ascending=1,method='min')
feature=all_data.groupby(['qid','day','hour'])['uid'].agg('count').reset_index()
feature.columns=['qid', 'day', 'hour', 'qday_sametime_icount']
all_data=all_data.merge(feature,on=['qid', 'day', 'hour'],how='left')

In [13]:
feature=all_data.groupby(['qid'])['time'].agg('nunique').reset_index()
feature.columns=['qid', 'qid_itime_nunique']
all_data=all_data.merge(feature,on=['qid'],how='left')

In [14]:
feature=all_data.groupby(['qid'])['time'].agg('count').reset_index()
feature.columns=['qid', 'qid_i_count']
all_data=all_data.merge(feature,on=['qid'],how='left')

In [15]:
all_data['qitime_ratio']=all_data['']/all_data['qid_i_count']

In [16]:
feature=all_data.groupby(['qid','day'])['time'].agg('nunique').reset_index()
feature.columns=['qid','day', 'qid_giday_time_nunique']
all_data=all_data.merge(feature,on=['qid','day'],how='left')

In [17]:
feature=all_data.groupby(['qid'])['qday_sametime_icount'].agg('max').reset_index()
feature.columns=['qid', 'qday_sametime_icount_max']
all_data=all_data.merge(feature,on=['qid'],how='left')

In [18]:
all_data['qitime_ratio2']=all_data['qid_itime_nunique']/all_data['qday_sametime_icount_max']

In [22]:
all_data=all_data.drop_duplicates(['qid','uid'])

In [23]:
all_data2=pd.concat([train[['qid','uid','time']],test2])

In [26]:
all_data2=all_data2.merge(all_data,on=['qid','uid'],how='left')

In [28]:
all_data2.drop(['time_x','time_y','day','hour','qid','uid'],axis=1).to_hdf('feature_today_invite_new2.h5', key='data')

In [20]:
####前7天收到的邀请

In [11]:
all_data_u=all_data.drop(['hour','qid'],axis=1)

In [12]:
all_data_u=pd.merge(all_data,all_data_u,on='uid',how='inner')

In [13]:
all_data_u['sub_time']=all_data_u['time_x'].astype(int)-all_data_u['time_y'].astype(int)
all_data_u['sub_day']=all_data_u['day_x'].astype(int)-all_data_u['day_y'].astype(int)

In [14]:
all_data_u=all_data_u[(all_data_u['sub_time']>0) & (all_data_u['sub_time']<7*24)]

In [15]:
all_data_u=all_data_u.groupby(['uid','qid']).agg({'sub_time':['mean','min','max','std'],
                                                  'sub_day':['mean','min','max','nunique','count','median'],
                                                 }).reset_index()

In [16]:
all_data_u.columns=['uid','qid','7isub_time_mean','7isub_time_min','7isub_time_max','7isub_time_std',
                   '7isub_day_mean','7isub_day_min','7isub_day_max','7isub_day_nunique','7isub_day_count','7isub_day_median']

In [17]:
import numpy as np
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [18]:
all_data_u=reduce_mem_usage(all_data_u)

Mem. usage decreased to 276.25 Mb (66.7% reduction)


In [21]:
all_data_u.to_hdf('u7days_invite_count2.h5', key='data')

In [20]:
all_data_u

,uid,qid,7isub_time_mean,7isub_time_min,7isub_time_max,7isub_time_std,7isub_day_mean,7isub_day_min,7isub_day_max,7isub_day_nunique,7isub_day_count,7isub_day_median
0,M1000000382,Q2418257178,37.000000,27,47,14.140625,1.500000,1,2,2,2,1.5
1,M1000000382,Q2557202265,47.000000,47,47,NaN,2.000000,2,2,1,1,2.0
2,M1000000382,Q3739876694,98.000000,27,144,51.093750,4.000000,1,6,4,4,4.5
3,M1000000382,Q3910864809,77.500000,54,101,33.218750,3.000000,2,4,2,2,3.0
4,M1000000382,Q4038278259,94.687500,70,117,23.593750,4.000000,3,5,3,3,4.0
5,M1000000382,Q4251023727,20.000000,20,20,NaN,1.000000,1,1,1,1,1.0
6,M1000000382,Q4280175724,85.500000,72,99,19.093750,3.500000,3,4,2,2,3.5
7,M1000000382,Q920413653,78.000000,21,120,51.187500,3.333984,1,5,3,3,4.0
8,M1000000983,Q1094985150,145.000000,145,145,NaN,6.000000,6,6,1,1,6.0
9,M1000000983,Q1425436837,35.500000,23,48,17.671875,1.500000,1,2,2,2,1.5


In [ ]:
####前7天收到的邀请

In [6]:
all_data_u=all_data.drop(['uid','time'],axis=1)

In [7]:
all_data_u=pd.merge(all_data_u,all_data_u,on='qid',how='inner')

In [8]:
all_data_u['sub_day']=all_data_u['day_x'].astype(int)-all_data_u['day_y'].astype(int)

In [9]:
all_data_u=all_data_u[(all_data_u['sub_day']>0) & (all_data_u['sub_day']<8)]

In [10]:
all_data_u=all_data_u.groupby(['qid','day_x'])['sub_day'].agg(['count','nunique']).reset_index()

In [11]:
all_data_u.columns=['uid','day','q7days_invite_pcount','q7days_invite_daynunique']

In [12]:
all_data_u['q7days_invite_pcount']=all_data_u['q7days_invite_pcount']/all_data_u['q7days_invite_daynunique']

In [14]:
all_data_u

,uid,day,q7days_invite_pcount,q7days_invite_daynunique
0,Q1000007604,3839,2.000000,1
1,Q1000007604,3840,1.500000,2
2,Q1000007604,3841,1.333333,3
3,Q1000007604,3842,2.500000,4
4,Q1000007604,3843,1.400000,5
5,Q1000008276,3862,3.000000,1
6,Q1000008276,3863,2.000000,2
7,Q1000008276,3867,3.333333,3
8,Q100002777,3849,22.000000,1
9,Q1000049860,3866,1.000000,1


In [16]:
#all_data_u.to_csv('q7days_invite_count.csv',index=False)
all_data_u.to_hdf('q7days_invite_count2.h5', key='data')

In [ ]:
###从ALL_DATA开始